# Notes for CSC3160 - Text Part

### 0. NLP Pyramid

自然语言处理的主要层面从底向上排列:

- 形态学(Morphology) : 处理单词的内部结构,如词根、词缀等。
- 句法(Syntax) : 分析句子的语法结构,理解词与词之间的关系。
- 语义(Semantics) : 理解句子和段落的含义和意义。
- 语用(Pragmatics) : 考虑语境、语意隐含等因素,理解语言在特定情况下的真正用途。


### 1.Text Processing and Regular Expression

#### 1.1 Corpus(语料库): tokens vs vocabulary

- Type： type 指的是在文本中不同实例，简单来说，就是独特的单词或词汇。

- Token：token 指的是某个 type 的实例。一个 type 可以多次出现在文本中，每一次出现就被称为一个 token。

#### 1.2 Text Normalization
文本归一化：指将文本标准化，转换为统一的格式，便于机器处理。这是 NLP 的基本步骤之一。归一化流程：
- Tokenizing（词语分割-也叫‘分词’）：将文本分割成单个的词或 token，并保留句子的意思。这一步是文本分析的基础。
  - Example ： I can't believe it's 2023 already!     ——————>  Tokens: ["I", "can't", "believe", "it's", "2023", “already!"]

- 规范化单词格式（Normalizing word formats）：包括将大写转换为小写，去掉标点符号，处理缩写等，确保所有词的格式一致。
- 句子分割（Sentence segmentation）：将文本分割成独立的句子，便于后续的分析。

#### 1.3 Subword Tokenization - 用于解决 Word tokenization 的 Out-Of-Vocabulary 问题
- Definition : 这是将文本拆分成比单词(words)更小的tokens的过程。子词(subwords)可以是任意字符串。这种方法允许模型处理不同形态和拼写变体的单词，增强了其对稀有单词和新词的处理能力

- Tokenization schemes:
  - Token learning: 这是一个过程，通过该过程模型学习如何有效地将输入文本划分为标记。通过从大规模语料库中学习，模型能够识别最有可能成为有效子词的字符序列。

  - Token segmenter: 这是一种工具或算法，用于根据特定标记化方案将文本划分成标记。它根据定义的规则或学习到的知识来决定如何划分输入文本。

- Algorithms: 
  - 字节对编码（Byte-Pair Encoding, BPE）
  - 单元语言建模（Unigram Language Modeling）
  - 词片（Wordpiece）: 在BERT时代，WordPiece 分词方法被广泛应用，比如 BERT、DistilBERT等

##### 1.3.1 Byte-Pair Encoding（BPE）
BPE 的核心思想是逐步合并出现频率最高的子词，从而构建出一个词汇表，以下是核心步骤：
- 1 计算初始词表：通过训练语料获得或者最初的英文中26个字母加上各种符号以及常见中文字符，这些作为初始词表。
- 2 构建频率统计：统计所有子词单元对（两个连续的子词）在文本中的出现频率。
- 3 合并频率最高的子词对：选择出现频率最高的子词对，将它们合并成一个新的子词单元，并更新词汇表。
- 4 重复合并步骤：不断重复步骤 2 和步骤 3，直到达到特定条件，如达到预定的词汇表大小、合并次数。
- 5 分词：使用最终得到的词汇表对文本进行分词。

BPE Algorithm:

```
function BYTE-PAIR ENCODING(strings C, number of merges `k`) returns vocab V

V ← all unique characters in C              # initial set of tokens is characters
for i = 1 to k do
    tL, tR ← Most frequent pair of adjacent tokens in C
    tNEW ← tL + tR                         # make new token by concatenating
    V ← V + tNEW                           # update the vocabulary
    Replace each occurrence of tL, tR in C with tNEW  # and update the corpus
return V
```

#### 1.4 String Similarity - How Similar Are Two Strings?

**Edit distance** : 使用 `edit operation` 将一个字符串变成另一个字符串需要几次操作。<p>
**Edit operations** : `Insertion(增加字符)` , `Deletion(删除字符)` , `Substitution(更新字符)`  <p>
**Minimum edit distance** : the minimum number of `editing operations` needed to transform one string into another. <p>
**Alignment** : The minimum edit distance can be represented as an `alignment`, and thus be computed. <p>
**Alignment Algorithm** : Suppose string `a` has M characters, string `b` has N characters. Define value $D(h,k)$ which represents the min edit distance of transforming string `h` to `k` and let `0` represents an `empty string`.
- 原理：Recurrence relation(递推关系)：
  - 对于字符串a的前i个字符，记其为字符串 $i$; 对于字符串b的前j个字符,记其为字符串$j$, 有以下2种情况:
    - 如果两个字符相同,则 $D(i,j) = D(i-1,j-1)$.
    - 如果两个字符不同,则 $D(i,j) = \min \{ D(i-1,j)+1, D(i,j-1)+1, D(i-1,j-1)+2 \}$, 这分别对应一下三种子情况：
      - 插入操作:在a中插入一个字符,编辑距离增加1
      - 删除操作:从a中删除一个字符,编辑距离增加1
      - 替换操作:将a的一个字符替换为b的对应字符,编辑距离增加2 （这里认为替换操作本质上是一个删除操作加插入操作，距离应该加2）
- 算法
```python

D[i,0] = i
D[0,j] = j    #Initialization


For i = 1...M:
    For j = 1...N:
        
        if x(i) == y(j):
            D[i,j] = D[i-1,j-1]
        else:
            D[i,j] = min(
                D[i-1,j] + 1,    # Deletion
                D[i,j-1] + 1,    # Insertion
                D[i-1,j-1] + 2   # Substitution
            )

D[N, M] is the distance   # Termination
```
#### 1.5 Regular Expression
**Definition** : A sequence of characters that specifies a pattern in text.正则表达式是一种用于匹配和操作文本的强大工具，它是由一系列字符和特殊字符组成的模式，用于描述要匹配的文本模式. 正则表达式可以在文本中查找、替换、提取和验证特定的模式.<p>

### 2. Words, Morphology, and Parts of Speech

#### 2.1 Words
- Words are at the interface between phonology, syntax and semantics
- Words are not atoms. They have internal structure

#### 2.2 Linguistic morphology
**Definition** : The study of how `morphemes(语素)` join together to form words. (单词由一个或多个 Morpheme 组成)
- Morphemes are the minimal units of linguistic form and meaning
- **Difference between words and morphemes** : between two words, we can usually insert some other words, while between two morphemes we can’t
- **Remark** : Whitespace is not always a good test for the word/morpheme distinction in English. Compound nouns are often spelled with whitespace between their components, yet they are a single word,like 'picture frame'



### 2. Words, Morphology, and Parts of Speech

#### Representations of The Meaning of Words